In [1]:
!pip install -q pandas tqdm 
!pip install -q transformers==4.55.0 # llm requires >=4.46.0
!pip install -q safetensors==0.4.3 # downgrade for torch 2.1.0
!pip install -q bitsandbytes==0.43.2 accelerate==1.9.0 # quantization


[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import re
import os
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

# Data Load & Define utils

In [2]:
# test = pd.read_csv('./test.csv')
test = pd.read_csv('../test.csv')

In [3]:
import re

# 선택지 라인: 1), 1., 1:, 1- , "1 " 등 유연 매칭
CHOICE_LINE = re.compile(r"^\s*(\d{1,2})\s*(?:[.):\-]|\s)\s*(.+)$")

def is_multiple_choice(question_text: str) -> bool:
    lines = question_text.strip().split("\n")
    option_count = sum(bool(CHOICE_LINE.match(l)) for l in lines)
    return option_count >= 2

def extract_question_and_choices(full_text: str):
    lines = full_text.strip().split("\n")
    q_lines, options = [], []
    for line in lines:
        m = CHOICE_LINE.match(line)
        if m:
            num, body = m.group(1), m.group(2).strip()
            options.append(f"{num} {body}")  # 번호 보존
        else:
            if line.strip():
                q_lines.append(line.strip())
    question = " ".join(q_lines)
    return question, options

In [4]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                "아래 질문에 대해 적절한 선택지를 출력하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                # "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                f"질문: {text}\n\n"
                # "답변:"
                )   
    return prompt

# Model Load

In [5]:
# 모델 선택
models = [
    "gemma-ko-7b", # baseline
    "ax-4.0-light-7b", # skt
    # "polyglot-12.8b",
    # "koalpaca-polyglot-12.8b",
    "midm-2.0-11.5b", # kt
    # "HyperCLOVAX-SEED-Think-14B", # naver
    # "kanana-1.5-15.7b-a3b-instruct", # kakao
    # "exaone-4.0-32b" # lg
]
selected_model = models[1]
# model_path = f"/workspace/models/{selected_model}" # 로컬 저장 모델 경로
model_path = f"/workspace/models/{selected_model}_qlora/merged"

# # 4bit 설정
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.bfloat16, # NaN 방지
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4"
# )

# # 8bit 설정
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,          # 4bit → 8bit
#     llm_int8_threshold=6.0,     # 기본값 (필요 시 조정)
#     llm_int8_has_fp16_weight=False  # True로 하면 일부 레이어 FP16 유지
# )

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    padding_side="left"
)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto", # GPU 자동 배정
    # quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    # trust_remote_code=True # naver
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Inference

In [22]:
import math

def sanitize_answer(x: str, is_mc: bool) -> str:
    # 타입/값 기반 결측 정리
    if x is None:
        return "" if not is_mc else "0"
    if isinstance(x, float) and math.isnan(x):
        return "" if not is_mc else "0"

    s = str(x).strip()
    if s.lower() in {"nan", "none", "null"}:
        return "" if not is_mc else "0"
    return s

def looks_like_nan_spam(s: str) -> bool:
    t = s.strip().lower()
    return bool(re.fullmatch(r'(nan[\s,]*)+', t))

def extract_answer_only(generated_text: str, original_question: str) -> str:
    # return_full_text=False면 보통 답만 오지만, 방어적으로 한 번 더 정리
    text = (generated_text or "").strip()
    if "답변:\n" in text:
        text = text.split("답변:\n")[-1].strip()
    elif "답변:" in text:
        text = text.split("답변:")[-1].strip()

    # 'nan,nan,...' 같은 비정상 생성 방지
    if looks_like_nan_spam(text):
        text = ""

    is_mc = is_multiple_choice(original_question)
    if is_mc:
        m = re.search(r"\b([1-9][0-9]?)\b", text)
        return m.group(1) if m else "0"   # 실패 시 "0"
    else:
        # 주관식은 너무 길면 잘라주기 (출력 폭주 방지)
        if not text:
            return ""
        # # 한 문단/한 줄 우선
        # text = text.split("\n\n")[0].split("\n")[0].strip()
        # if len(text) > 120:
        #     text = text[:120].rstrip()
        # 'nan', 'none', 'null' 같은 값은 빈문자 처리
        if text.lower() in {"nan", "none", "null"}:
            return ""
        return text


In [7]:
# pad/eos 안전장치
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # 배치 패딩 안정화 (causal LM에서 권장)

In [23]:
from tqdm import tqdm
from transformers import pipeline

batch_size = 8

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    batch_size=batch_size
)

questions = test['Question'].tolist()
prompts   = [make_prompt_auto(q) for q in questions]

# 인덱스 분리
mc_idx, sa_idx = [], []
for i, q in enumerate(questions):
    (mc_idx if is_multiple_choice(q) else sa_idx).append(i)

def build_inputs(idxs):
    # 여기서만 답변: (공백 포함) 붙임
    return [prompts[i].rstrip() + "\n\n답변: " for i in idxs]

preds = [""] * len(prompts)

# 4-1) 객관식: 그리디 + 짧게 (루프/스팸 방지)
if mc_idx:
    mc_inputs = build_inputs(mc_idx)
    mc_outputs = pipe(
        mc_inputs,
        max_new_tokens=6,
        do_sample=False, temperature=None,
        no_repeat_ngram_size=3,
        repetition_penalty=1.06,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        return_full_text=False
    )
    for i, out in zip(mc_idx, mc_outputs):
        gen = out[0]["generated_text"]
        # 디버그 (필요 시)
        print("[MC out]", gen)
        preds[i] = extract_answer_only(gen, questions[i])

# 4-2) 주관식: 짧은 서술 (100자 내외)
if sa_idx:
    sa_inputs = build_inputs(sa_idx)
    sa_outputs = pipe(
        sa_inputs,
        max_new_tokens=512,              
        # do_sample=False, temperature=None,
        do_sample=True, temperature=0.2, top_p=0.9,
        no_repeat_ngram_size=3,
        repetition_penalty=1.10,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        return_full_text=False
    )
    for i, out in zip(sa_idx, sa_outputs):
        gen = out[0]["generated_text"]
        # 디버그 (필요 시)
        print("[SA out]", gen)
        preds[i] = extract_answer_only(gen, questions[i])

# (선택) 화면 확인 시 NaN 문자열 보이지 않게
# -> preds는 전부 문자열이므로 보통 NaN이 생기지 않습니다.
# 결과 합치기 예:
# out_df = pd.DataFrame({"Question": questions, "Pred": pd.Series(preds, dtype="string")})
# print(out_df.to_string(na_rep=""))
# out_df.to_csv("preds.csv", index=False)


Device set to use cuda:0


[MC out] 1, 소비자금융업은 금융투자
[MC out] 1 수행인원
위험 관리
[MC out] 1 정보보호및개인정보보호정책의
[MC out] 3, 개인정보 파기절차

[MC out] 4 통화신용정책의 수행
[MC out] 2 법정대리의 동의를 받아야
[MC out] 
nan,nan,1
[MC out] 3 전자금융지주업자의등록
[MC out] 1 SPF(이메일 발신자의
[MC out] 4, 모니터링 통제  

[MC out] 3, 대칭 키는 two 개의
[MC out] 4 계좌 정보는 범죄에 이용 될
[MC out] 3, 네트워크 트래픽를 분석하여
[MC out] 4, 국내대리인의 지
[MC out] 1, 신용회복support 협약
[MC out] 4, 정보주인의 권리
[MC out] 1 정보보호최저책임자는전자
[MC out] 1 국내 대리인
국내 대리
[MC out] 4 법에 의한 경우
개인정보
[MC out] 1 신용정보의 제공능력 향상
[MC out] 3 고객 만족도와 관련된 기준
[MC out] 5 IP주소의 사설화와
[MC out] 2 파일 전달을 위한 프로토
[MC out] 3, 평가에 불이익와 혜택을
[MC out] 3 불빛이나 소리, 또는
[MC out] 3 서버 하드웨 장애 자동
[MC out] 1, 데이터베이스의 기밀이
[MC out] 1 공공정보의 장
[MC out] 3 법인또는개인이위반행위를
[MC out] 3 영업 정지
전자금융
[MC out] 1, 위원이 해당하는 사안에
[MC out] 1, 제15조와 제
[MC out] 1 코드는 보안 취약점이
[MC out] 2 정보의무단수집 으로 인한
[MC out] 2 금융감독원이 사전 심의를 해야
[MC out] 4 암호화한 데이터 복호
[MC out] 1, 7year 이하의
[MC out] 4, 침해사고 응답 및
[MC out] 1, 에러 메시스에 시스템
[MC out] 5, 원격접속 시에 다중
[MC out] 3 서명자가 실지명을
[MC out] 4 온라인 송신
4,
[MC out] 4

In [24]:
preds

['1',
 '1',
 '1',
 '3',
 '트로이 목마 기반의 원격제어 멀웨어(Remote Access Trojans, RATs)는 사이버 공격에서 매우 흔하게 사용되는 도구 중 하나입니다. 이러한 악성코드는 주로 사용자의 시스템에 침투하여 원격으로 제어할 수 있는 기능을 제공합니다. 아래에서 트로이 목마의 특징 및 주요 탐지 지표에 대해 자세히 설명하겠습니다.\n\n\n### 1. 트로이목마의 특징\n\n#### a. 위장 기술\n- **정상적인 소프트웨어처럼 보이기**: 트로이목은 종종 합법적인 프로그램이나 파일과 유사하게 위장합니다. 예를 들어, 게임 패치나 유용한 유틸리티로 가장할 수 있습니다.\n- 사용자 신뢰를 유도하기 위해 친숙한 이름이나 아이콘을 사용합니다.\n  \n#### b. 백그라운드 실행\n- 트로이마는 사용자가 프로그램을 실행할 때 자동으로 백그라운드에서 작동하며, 시스템 리소스를 소모하지 않도록 설계되어 있습니다. 이는 사용자에게 감지되지 않게 하기 위함입니다.\n\n\n#### c. 다양한 기능 제공\n- 원격 제어 기능 외에도 데이터 수집, 화면 캡처, 키 입력 기록 등 다양한 기능을 수행할 수 있습니다..\n- 특정 서버와의 통신을 통해 추가 명령어를 수신하고 이를 수행함으로써 공격자의 제어를 받을 수 있습니다.,\n\n### 2. 주요 탐지지표\n\n트로이와 같은 RAT를 탐지하기 위해서는 다음과 같은 여러 가지 지표를 활용할 수 있습니다.:\n\n##### a. 비정상적인 네트워크 활동\n- 의심스러운 IP 주소로의 지속적인 연결 시도 또는 대량의 데이터 전송이 발생할 경우, 이는 RAT가 외부와 통신하고 있다는 신호일 수 있습니다:.\n\n- 포트 스캔이나 비정상적인 트래픽 패턴이 발견될 경우, 추가적인 조사가 필요합니다.\n  \n##### b. 시스템 성능 저하\n- CPU 사용률 증가, 메모리 사용량 급증 등의 시스템 성능 저하는 RAT의 존재를 나타낼 수 있습니다:,\n\n이러한 성능 저하의 원인이 정상적인 작업

# Submission

In [25]:
# sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission = pd.read_csv('../sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv(f'../output/submission_{selected_model}.csv', index=False, encoding='utf-8-sig')